In [102]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import os
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

In [103]:
# Set environment variables for reproducibility
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True, warn_only=True)

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

torch.manual_seed(40)
torch.cuda.manual_seed_all(40)

In [104]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes, use_dropout=False,acti1=nn.ReLU,acti2=nn.ReLU,norm_type='none'):
        super().__init__()
        self.norm_type=norm_type.lower() if norm_type else None
        def norm_layer(channels, shape=None, location='conv'):
            if norm_type == 'batch':
                return nn.BatchNorm2d(channels) if location == 'conv' else nn.BatchNorm1d(channels)
            elif norm_type == 'layer':
                return nn.LayerNorm(shape)
            elif norm_type == 'instance':
                return nn.InstanceNorm2d(channels) if location == 'conv' else nn.InstanceNorm1d(channels)
            else:
                return nn.Identity()
        self.model=nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,padding=1),
            norm_layer(32, location='conv'),
            acti1(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,kernel_size=3, padding=1),
            norm_layer(64, location='conv'),
            acti1(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*32*32,256),
            norm_layer(256, location='fc'),
            acti2(),
            nn.Dropout(0.4 if use_dropout else 0.0),
            nn.Linear(256,num_classes)
        )
    def forward(self,x):
        return self.model(x)

In [106]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0

    for imgs, labels in dataloader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    return running_loss / len(dataloader.dataset), correct / len(dataloader.dataset)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0

    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    return running_loss / len(dataloader.dataset), correct / len(dataloader.dataset)


In [107]:
def get_transforms(use_normalization=True):
    transform_list = [
        transforms.Resize((128, 128)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
    ]
    if use_normalization:
        transform_list.append(
            transforms.Normalize(mean=[0.5], std=[0.5])
        )
    return transforms.Compose(transform_list)

In [108]:
def get_scheduler(optimizer, name='step', step_size=5, gamma=0.5):
    if name == 'step':
        return optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    elif name == 'cosine':
        return optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    elif name == 'none':
        return None
    else:
        raise ValueError(f"Unknown scheduler: {name}")

In [109]:
def run_training(root='./data',
                batch_size=32, 
                num_epochs=10, 
                lr=0.001, 
                use_dropout=True,
                use_normalization=True,
                scheduler_name='step',
                norm_type='batch',
                acti1=nn.ReLU,acti2=nn.ReLU):
    transform = get_transforms(use_normalization)
    dataset = datasets.Caltech101(root='./data', download=False, transform=transform)
    num_classes = len(dataset.categories)

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    model=CNNModel(num_classes=num_classes, use_dropout=use_dropout,norm_type=norm_type,acti1=acti1,acti2=acti2).to(device)
    criterion=nn.CrossEntropyLoss()
    optimizer=optim.Adam(model.parameters(),lr=lr)
    scheduler=get_scheduler(optimizer, name=scheduler_name)
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)

        if scheduler:
            scheduler.step()

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc*100:.4f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.4f}%\n")

In [110]:
run_training(
    use_dropout=False,
    use_normalization=True,
    scheduler_name='step',
    num_epochs=5,
    norm_type='batch',
    acti1=nn.ReLU,
    acti2=nn.ELU
)

Epoch 1/5:
  Train Loss: 2.5922, Accuracy: 44.3164%
  Val   Loss: 2.0039, Accuracy: 56.2788%

Epoch 2/5:
  Train Loss: 1.0962, Accuracy: 77.3664%
  Val   Loss: 1.6777, Accuracy: 59.9078%

Epoch 3/5:
  Train Loss: 0.1864, Accuracy: 97.9974%
  Val   Loss: 1.5979, Accuracy: 62.6152%

Epoch 4/5:
  Train Loss: 0.0263, Accuracy: 99.8847%
  Val   Loss: 1.4764, Accuracy: 64.9194%

Epoch 5/5:
  Train Loss: 0.0110, Accuracy: 99.9568%
  Val   Loss: 1.4550, Accuracy: 65.6682%

